<h1>Motivation</h1>
<p>This notebook is designed to analyze the description of every injury that is recorded as Sprain/strain InjuryIllnessType to see if there are some trends that we can find.</p>

# Import the libraris to read and manipulate data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [2]:
data = pd.read_excel('Falls Same Level ML data.xlsx')
data.head()

,Organization,Sub-Organization,Site,System Case ID,Case Date,Incident Type,Body Part,Cause Object,Injury Type,Description,Descriptive Root Cause,Corrective Actions,Case Level
0,Amazon Transportation Services,Sort Center - North America,MSP5,245,2017-11-14,"Falls, same level",Leg,Equip: Pallet,Fracture (Non-Hairline),"MSP5 (Right ankle fracture, slip/trip/fall, Pa...",AA did not have their on path or task while wo...,AA should get coached on taking two packages a...,Level B
1,North America Fulfillment Centers (FCs),Returns,CVG2,2662,2017-11-15,"Falls, same level",Elbow,Debris: Other,Fracture (Non-Hairline),On 11/15/17 at approx 11:00 AA was working in ...,It was determined the AA unintentionally tripp...,Perform FSI with focus on lose banding. ASC me...,Level B
2,Europe Fulfillment Centers (FCs),AR Sortable,MAN1,409,2017-11-14,"Falls, same level",Knee,NaN,Loss of consciousness,Amazon Associate (AA) has been staffed on AFE ...,AA is not feeling well.,AM to ensure that they inform AA if they are n...,Level B
3,Amazon Transportation Services,Sort Center - North America,BOS5,612,2017-11-17,"Falls, same level",Shoulder,NaN,Loss of consciousness,AA was scanning on line G @ I-Beam G-16. AA pa...,"AA had been feeling ill during the day, they w...",none needed non-occ injury,Level B
4,Amazon Transportation Services,Sort Center - North America,LAS5,53,2017-11-15,"Falls, same level",Head - Facial Area,NaN,Loss of consciousness,"On Nov 15, 2017, at approximately 0826 hours, ...",AA fell on the floor and hit their head.,To remind AA to be mindful of their surroundings.,Level B


In [4]:
data.shape

(1028, 13)

## Time to preprocess the text. 
<p>I will use the libraries __`spaCy`__ and __`re`__ for this purpose.</p>


In [3]:
import spacy
nlp = spacy.load('en')

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [16]:
#prevent DeprecationWarnings RuntimeWarnings and UserWarnings from showing
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)
warnings.filterwarnings("ignore",category=UserWarning)

In [23]:
import re
data['RawPrep'] = data.Description.apply(lambda desc: [word.text.lower() for word in nlp(desc) if word.pos_ not in [ 'ADV', 'ADP', 'DET', 'PART', 'PRON', 'PUNCT']])

In [38]:
def remove_recurring_words(sent, is_list = False):
    if is_list:
        tokens = sent
    else:
        tokens = sent.split()
    return ' '.join([word for index, word in enumerate(tokens) if not tokens[index] == tokens[index - 1]])
remove_recurring_words('here he is saying saying all men men are are equal equal')

'here he is saying all men are equal'

In [ ]:
spacy.explain('pos_')

## Removing Stop-Words:
Stopwords are words that occour too frequently in a text corpus but which do not carry a lot of meaning. I will use the set of stopwords from `nltk`. Also, since aa in our corpus represents the word associate, I will replace all occurreces of the word aa with the word associate.

<p> I will also remove words with less than 3 characters.

In [24]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')
print(my_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [25]:
data['RawPrep'] = data.RawPrep.apply(lambda x: [word for word in x if (not word in my_stopwords and len(word) >2)])

## Time to perform some Topic modelling.
Let me guess that the number of topics in these documents (descriptions) is equal tot he number of principal body part.

In [26]:
data.head(2)

,Organization,Sub-Organization,Site,System Case ID,Case Date,Incident Type,Body Part,Cause Object,Injury Type,Description,Descriptive Root Cause,Corrective Actions,Case Level,RawPrep
0,Amazon Transportation Services,Sort Center - North America,MSP5,245,2017-11-14,"Falls, same level",Leg,Equip: Pallet,Fracture (Non-Hairline),"MSP5 (Right ankle fracture, slip/trip/fall, Pa...",AA did not have their on path or task while wo...,AA should get coached on taking two packages a...,Level B,"[msp5, right, ankle, fracture, slip, trip, fal..."
1,North America Fulfillment Centers (FCs),Returns,CVG2,2662,2017-11-15,"Falls, same level",Elbow,Debris: Other,Fracture (Non-Hairline),On 11/15/17 at approx 11:00 AA was working in ...,It was determined the AA unintentionally tripp...,Perform FSI with focus on lose banding. ASC me...,Level B,"[11/15/17, approx, 11:00, working, carpets, ar..."


In [27]:
data['Body Part'].nunique()

22

## Topic modelling - GENSIM Library.
I first import the library and prepare a corpus dictionary, then I create a bag of words and apply the lda model on the bag of words.

 <h3> The steps to create a gensim topic model are:</h3>
 <ul>
    <li>Create a corpus dictionary that stores words together with their indexes using the `Dictionary` method of `gensim.corpora` module.</li>
    <li>Store the list of tokenized documents. Since each tokenized document is a list of words, you have a list of list of words.</li>
    <li>Create a count corpus that is a list of lists of tuples (word indexes, frequecy) for each word in the document bag of words.</li>
</ul>

In [28]:
import gensim.corpora as corpora
import gensim
from gensim.models import CoherenceModel
# Create Dictionary
id2word = corpora.Dictionary(data.RawPrep.values.tolist())

# Create Corpus
texts = data.RawPrep.values.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View the first document corpus
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 4), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 4), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 6), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 2), (69, 1), (70, 3), (71, 1), (72, 5), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 1), (97, 1), (98, 1), (99, 4), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 10), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

In [29]:
id2word[0]

'11/14/17'

Sometimes, it is really important to have human readable format of corpus (word, frequency). We check the (word, frequency) corpus for the first document below.

In [30]:
print([[(id2word[ID], freq) for ID, freq in cp] for cp in corpus[:1]])

[[('11/14/17', 1), ('11/15/17', 1), ('11/21/17', 1), ('11/22/17', 1), ('1904.7', 1), ('300', 1), ('7/2/18', 1), ('accommodations', 3), ('amcare', 1), ('ankle', 4), ('area', 1), ('attempts', 1), ('attention', 1), ('aware', 1), ('blocked', 1), ('calls', 1), ('case', 4), ('cases', 1), ('cfr', 1), ('communication', 2), ('communications', 2), ('constant', 1), ('continue', 1), ('correct', 1), ('criteria', 1), ('day', 1), ('distracted', 1), ('documentation', 2), ('electronic', 1), ('ensure', 2), ('esd', 1), ('established', 1), ('eyes', 1), ('failed', 2), ('fall', 1), ('foot', 1), ('fracture', 2), ('general', 1), ('grabbing', 1), ('holding', 1), ('identified', 1), ('information', 2), ('initial', 1), ('injured', 1), ('injury', 6), ('investigation', 1), ('jsa', 1), ('label', 1), ('lack', 1), ('land', 1), ('lane', 1), ('line', 2), ('loa', 1), ('location', 1), ('locations', 1), ('log', 1), ('made', 2), ('make', 1), ('making', 1), ('manager', 1), ('manner', 1), ('medical', 3), ('moved', 1), ('movin

# Fit a topic model and visualize the topics
My favorite visualization for topic models is `pyLDAvis.` Make sure to import pyLDAvis.gensim as well.

In [31]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # This import is neccessary for pyLDAvis to be used in python.
import matplotlib.pyplot as plt


In [35]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=22,
                                           random_state=100,
                                           chunksize=2,
                                           passes = 5)#,
#                                            update_every=100,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.366221 -0.110601       1        1  13.051451
11     0.111519  0.287824       2        1  11.939251
13     0.256528 -0.143283       3        1   8.149836
15     0.068249  0.204005       4        1   7.593796
9      0.028301  0.111895       5        1   4.578054
2      0.008473  0.151859       6        1   4.230257
12    -0.052439 -0.018548       7        1   4.220196
19     0.127165 -0.108951       8        1   3.982491
6     -0.067363 -0.018638       9        1   3.700565
16     0.024565  0.112823      10        1   3.472712
7     -0.070970 -0.033463      11        1   3.357744
14    -0.057735  0.014516      12        1   3.340766
5     -0.079074 -0.055239      13        1   3.275869
3     -0.081038 -0.048130      14        1   3.244074
1     -0.055132 -0.042708      15        1   3.164180
21    -0.059247 -0.026509      16        1   2.998007
17    -0.079914 -0.037396      17        1   2.826034
20    -0.081513 -0.052002      18        1   2.806298
18    -0.089205 -0.046297      19        1   2.753212
8     -0.038904 -0.041848      20        1   2.742663
4     -0.085818 -0.040481      21        1   2.451407
10    -0.092668 -0.058828      22        1   2.121139, topic_info=     Category         Freq              Term        Total  loglift  logprob
term                                                                       
181   Default  1750.000000         associate  1750.000000  30.0000  30.0000
267   Default  1422.000000          incident  1422.000000  29.0000  29.0000
72    Default   636.000000            pallet   636.000000  28.0000  28.0000
71    Default  1295.000000              pain  1295.000000  27.0000  27.0000
230   Default   582.000000            stated   582.000000  26.0000  26.0000
294   Default   849.000000              time   849.000000  25.0000  25.0000
8     Default  1169.000000            amcare  1169.000000  24.0000  24.0000
99    Default   924.000000             right   924.000000  23.0000  23.0000
145   Default   655.000000             level   655.000000  22.0000  22.0000
138   Default   479.000000             floor   479.000000  21.0000  21.0000
454   Default   444.000000           walking   444.000000  20.0000  20.0000
144   Default   736.000000              left   736.000000  19.0000  19.0000
415   Default   425.000000           slipped   425.000000  18.0000  18.0000
1226  Default   390.000000           trailer   390.000000  17.0000  17.0000
162   Default   681.000000             shift   681.000000  16.0000  16.0000
119   Default   748.000000           working   748.000000  15.0000  15.0000
136   Default   355.000000              felt   355.000000  14.0000  14.0000
118   Default   934.000000              work   934.000000  13.0000  13.0000
59    Default   362.000000           manager   362.000000  12.0000  12.0000
25    Default   475.000000               day   475.000000  11.0000  11.0000
135   Default   730.000000              fell   730.000000  10.0000  10.0000
81    Default   471.000000           process   471.000000   9.0000   9.0000
44    Default   559.000000            injury   559.000000   8.0000   8.0000
91    Default   765.000000          reported   765.000000   7.0000   7.0000
291   Default   329.000000           station   329.000000   6.0000   6.0000
593   Default   391.000000              task   391.000000   5.0000   5.0000
115   Default   391.000000           tripped   391.000000   4.0000   4.0000
198   Default   293.000000             first   293.000000   3.0000   3.0000
385   Default   458.000000              knee   458.000000   2.0000   2.0000
166   Default   503.000000         treatment   503.000000   1.0000   1.0000
...       ...          ...               ...          ...      ...      ...
1569  Topic22    69.491241          informed    70.346771   3.8410  -3.3415
6975  Topic22    67.477783            steady    68.557495   3.8373  -3.3709
1

#### Print the top 20 Keyword in each of the 22 topics with the importance of each word.

In [33]:
import pprint
for topic in lda_model.show_topics(num_words=30, num_topics=22, formatted = False):
    print('Topic {}:'.format(topic[0] +1), '\n', ' | '.join([wordpair[0] for wordpair in topic[1]]), '\n', '--'*78)
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))

Topic 1: 
 incident | left | shift | work | working | fell | pain | reported | aas | area | time | safety | foot | fall | one | report | wrist | hand | causing | lost | got | sort | walked | empty | end | start | caught | trip | jack | initial 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 2: 
 ground | two | able | seen | scheduled | provider | hospital | given | called | action | diagnosed | resume | occupational | contusion | assessment | duties | believed | stacked | minute | statements | cases | lifting | bring | send | lumbar | incline | gate | transported | tour | run 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 3: 
 amcare | right | ankle | leg | side | receive | time | week | condition | incident | lane | injured | associated | hour | hazards 

<h1>How to inteprete the topics:</h1>

<p> Each topic is made of key words or __sailient terms__ which are the words that carry the most meaning in that topic. The number besides each word is a measure of the importance of the word to that topic. These numbers are also called the __weights__ and they follow a probability distribution on the topic and so the sum of the weights for each topic is unity.</p>
<p> The topics are not printed in any order.</p>
<p> We use the topic coherence and perplexity to determin which how good our topic model is. I have printed the coherence and perplexity scores for our topic model below.</p>

In [22]:
# Compute Perplexity """lower the better."""
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score  """ Higher is better """
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v', topn=30)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("Topics_Coherence, Standard_Deviation, Support")# support is the topn # of words in the topic
coherence_model_lda.get_coherence_per_topic(with_std=True, with_support=True)



Perplexity:  -10.068086583130835

Coherence Score:  0.4146135265124346
Topics_Coherence, Standard_Deviation, Support


[(0.2440833649131661, 0.12300096157029075, 30),
 (0.4634541621150391, 0.14072369149344105, 30),
 (0.22638086854834688, 0.16359397634009898, 30),
 (0.39713846123961344, 0.1243953427114668, 30),
 (0.5013426394414733, 0.16706778347827428, 30),
 (0.534928849568034, 0.1510481945584095, 30),
 (0.3664589689718833, 0.17193480112180293, 30),
 (0.4021998914164192, 0.16584015563881066, 30),
 (0.5270496096727609, 0.14663945793875288, 30),
 (0.281450118154147, 0.20612950769423863, 30),
 (0.5536810397775587, 0.12024564527038337, 30),
 (0.20876202420952153, 0.18552437672700356, 30),
 (0.5408933531662459, 0.15966557023016306, 30),
 (0.4149177992600532, 0.36760817830831943, 30),
 (0.4880793346165662, 0.19157993185678235, 30),
 (0.22169478713990645, 0.21176907387993285, 30),
 (0.3248607938122203, 0.19537345700543127, 30),
 (0.5111557864780025, 0.1617531413082343, 30),
 (0.5421116683810538, 0.12861146644671667, 30),
 (0.30201887479861583, 0.20324743909240062, 30),
 (0.5343670824838598, 0.1416252161855040

In [39]:
l22 = []
for topic in lda_model.show_topics(num_words=20, num_topics=22, formatted = False):
    l22.append(('Topic {}:'.format(topic[0] +1), ' | '.join([wordpair[0] for wordpair in topic[1]])))
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))

In [40]:
pd.DataFrame(l22, columns= 'TopicNum KeyWords'.split()).to_excel('Topics22FallsSameLevel.xlsx', index = False)

## It seems as if 22 was too large a number for the number of topics. Let's change the number to 17 and see if we get better results.

In [41]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=17,
                                           random_state=100,
                                           chunksize=2,
                                           passes=10)#,
#                                            update_every=100,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.390054 -0.007861       1        1  23.598104
16     0.230862 -0.034301       2        1  10.810995
3     -0.006803  0.360114       3        1   8.498718
1      0.056091 -0.067334       4        1   5.902837
4     -0.055294 -0.021907       5        1   5.226067
14     0.020805 -0.046537       6        1   5.163571
5     -0.036836 -0.044192       7        1   4.809777
12    -0.040779  0.233434       8        1   4.744089
9     -0.071053 -0.045238       9        1   3.965228
13    -0.060553 -0.031076      10        1   3.930922
10    -0.065347 -0.052121      11        1   3.833462
2     -0.030275 -0.027517      12        1   3.586336
6     -0.076887 -0.057454      13        1   3.579979
11    -0.037006 -0.011063      14        1   3.455828
8     -0.064266 -0.042253      15        1   3.094540
0     -0.073779 -0.054663      16        1   2.996199
15    -0.078933 -0.050031      17        1   2.803338, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
181   Default  1753.000000   associate  1753.000000  30.0000  30.0000
267   Default  1363.000000    incident  1363.000000  29.0000  29.0000
230   Default   675.000000      stated   675.000000  28.0000  28.0000
72    Default   662.000000      pallet   662.000000  27.0000  27.0000
71    Default  1403.000000        pain  1403.000000  26.0000  26.0000
294   Default   846.000000        time   846.000000  25.0000  25.0000
8     Default  1161.000000      amcare  1161.000000  24.0000  24.0000
118   Default   956.000000        work   956.000000  23.0000  23.0000
454   Default   432.000000     walking   432.000000  22.0000  22.0000
25    Default   436.000000         day   436.000000  21.0000  21.0000
1226  Default   384.000000     trailer   384.000000  20.0000  20.0000
99    Default   913.000000       right   913.000000  19.0000  19.0000
162   Default   677.000000       shift   677.000000  18.0000  18.0000
59    Default   345.000000     manager   345.000000  17.0000  17.0000
44    Default   555.000000      injury   555.000000  16.0000  16.0000
144   Default   757.000000        left   757.000000  15.0000  15.0000
115   Default   350.000000     tripped   350.000000  14.0000  14.0000
593   Default   314.000000        task   314.000000  13.0000  13.0000
119   Default   782.000000     working   782.000000  12.0000  12.0000
136   Default   340.000000        felt   340.000000  11.0000  11.0000
81    Default   425.000000     process   425.000000  10.0000  10.0000
198   Default   288.000000       first   288.000000   9.0000   9.0000
166   Default   531.000000   treatment   531.000000   8.0000   8.0000
145   Default   733.000000       level   733.000000   7.0000   7.0000
135   Default   759.000000        fell   759.000000   6.0000   6.0000
91    Default   811.000000    reported   811.000000   5.0000   5.0000
314   Default   301.000000        dock   301.000000   4.0000   4.0000
184   Default   232.000000       break   232.000000   3.0000   3.0000
10    Default   579.000000        area   579.000000   2.0000   2.0000
291   Default   329.000000     station   329.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
59    Topic17   344.998749     manager   345.832581   3.5719  -2.0180
2227  Topic17    66.149193       fluid    66.983040   3.5618  -3.6696
1008  Topic17    58.831001        seek    59.664845   3.5603  -3.7869
707   Topic17    52.932480   equipment    53.766323   3.5587  -3.8925
1143  Topic17    51.179859         hip    52.013702   3.5582  -3.9262
712   Topic17    49.074059         pit    49.907902   3.5575  -3.9682
739   Topic17    46.273472     related    47.107315   3.5565  -4.0270
736   Topic17    45.767422         ice    46.601265   3.5563  -4.0380
445   Topic17    44.236099        icqa    45.069950   3.5557 

In [42]:
# Compute Perplexity """lower the better."""
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score  """ Higher is better """
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v', topn=30)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("Topics_Coherence, Standard_Deviation, Support")# support is the topn # of words in the topic
coherence_model_lda.get_coherence_per_topic(with_std=True, with_support=True)


Perplexity:  -9.411343091010684

Coherence Score:  0.3907127960635583
Topics_Coherence, Standard_Deviation, Support


[(0.5327470976850844, 0.16437757014167417, 30),
 (0.33520386410954917, 0.18832461867521705, 30),
 (0.41269188370167564, 0.13678022546364357, 30),
 (0.18157605058131313, 0.39895101573677155, 30),
 (0.4695453283881166, 0.17502851010040435, 30),
 (0.3965092478077512, 0.17237452576095355, 30),
 (0.4063711733464564, 0.16577750288410018, 30),
 (0.24104416534721324, 0.17912535416274497, 30),
 (0.4616857045816036, 0.13667950754842267, 30),
 (0.3634543055036721, 0.15948690923693062, 30),
 (0.5320866634057754, 0.13576797613386363, 30),
 (0.45946257797963763, 0.1423227533136382, 30),
 (0.36271081686225287, 0.2857527387562141, 30),
 (0.35660398880011873, 0.12958625287192982, 30),
 (0.3488931903222042, 0.14764379670727681, 30),
 (0.5370882790874805, 0.14113163662105835, 30),
 (0.24444319557058589, 0.2801570973468131, 30)]

In [43]:
l17 = []
for topic in lda_model.show_topics(num_words=30, num_topics=17, formatted = False):
    l17.append(('Topic {}:'.format(topic[0] +1), ' | '.join([wordpair[0] for wordpair in topic[1]])))
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))
for topic in lda_model.show_topics(num_words=30, num_topics=17, formatted = False):
    print('Topic {}:'.format(topic[0] +1), '\n', ' | '.join([wordpair[0] for wordpair in topic[1]]), '\n', '--'*78)
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))
pd.DataFrame(l22, columns= 'TopicNum KeyWords'.split()).to_excel('Topics17FallsSameLevel.xlsx', index = False)

Topic 1: 
 walking | break | back | require | morning | car | needed | works | obtained | spilled | designated | curb | good | know | says | nearby | injuring | multiple | legs | surface | saturday | bike | cement | stop | 26th | someone | ribs | makes | stumbled | bit 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 2: 
 proper | date | contact | returned | hand | school | came | treatment | ops | level | scanner | initial | regular | post | treated | desk | using | weight | hit | finished | detergent | lunch | ehs | involved | shift | safe | exit | 4/10 | bottle | helped 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 3: 
 conveyor | standard | knees | sort | cause | green | mile | trip | continue | pre | factors | aa2 | belt | environmental | performed |

In [47]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           chunksize=2,
                                           passes=8)#,
#                                            update_every=100,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.344393  0.015269       1        1  21.627501
5      0.119270 -0.189012       2        1  14.324885
0      0.063232 -0.076415       3        1  12.535598
4      0.064664  0.295175       4        1  12.023582
3     -0.110252 -0.081507       5        1   8.688878
9     -0.057732 -0.071013       6        1   8.097029
6     -0.091293 -0.031395       7        1   6.391324
1     -0.100563  0.190230       8        1   5.785262
8     -0.127762 -0.036600       9        1   5.581395
2     -0.103956 -0.014731      10        1   4.944542, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
181   Default  1639.000000   associate  1639.000000  30.0000  30.0000
267   Default  1365.000000    incident  1365.000000  29.0000  29.0000
71    Default  1311.000000        pain  1311.000000  28.0000  28.0000
230   Default   712.000000      stated   712.000000  27.0000  27.0000
294   Default   849.000000        time   849.000000  26.0000  26.0000
8     Default  1084.000000      amcare  1084.000000  25.0000  25.0000
72    Default   790.000000      pallet   790.000000  24.0000  24.0000
1226  Default   431.000000     trailer   431.000000  23.0000  23.0000
138   Default   549.000000       floor   549.000000  22.0000  22.0000
91    Default   760.000000    reported   760.000000  21.0000  21.0000
81    Default   429.000000     process   429.000000  20.0000  20.0000
145   Default   695.000000       level   695.000000  19.0000  19.0000
162   Default   653.000000       shift   653.000000  18.0000  18.0000
118   Default   901.000000        work   901.000000  17.0000  17.0000
119   Default   711.000000     working   711.000000  16.0000  16.0000
99    Default   880.000000       right   880.000000  15.0000  15.0000
593   Default   356.000000        task   356.000000  14.0000  14.0000
160   Default   432.000000      safety   432.000000  13.0000  13.0000
314   Default   298.000000        dock   298.000000  12.0000  12.0000
385   Default   397.000000        knee   397.000000  11.0000  11.0000
166   Default   519.000000   treatment   519.000000  10.0000  10.0000
198   Default   282.000000       first   282.000000   9.0000   9.0000
909   Default   358.000000         aas   358.000000   8.0000   8.0000
291   Default   332.000000     station   332.000000   7.0000   7.0000
10    Default   538.000000        area   538.000000   6.0000   6.0000
424   Default   329.000000     trained   329.000000   5.0000   5.0000
454   Default   441.000000     walking   441.000000   4.0000   4.0000
25    Default   438.000000         day   438.000000   3.0000   3.0000
105   Default   340.000000        site   340.000000   2.0000   2.0000
135   Default   694.000000        fell   694.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1156  Topic10   103.608192   condition   104.421425   2.9991  -3.7884
989   Topic10    95.972702        care    96.785172   2.9985  -3.8650
50    Topic10    89.277824        lane    90.090256   2.9978  -3.9373
509   Topic10    86.294243     scanner    87.106705   2.9975  -3.9713
734   Topic10    70.516899     hazards    71.329933   2.9954  -4.1732
67    Topic10    69.781105       order    70.593689   2.9953  -4.1837
520   Topic10    67.482651      weight    68.295113   2.9949  -4.2171
1008  Topic10    55.132328        seek    55.944885   2.9923  -4.4193
2235  Topic10    54.354237       stool    55.166744   2.9920  -4.4335
478   Topic10    53.992626         ehs    54.804977   2.9920  -4.4402
494   Topic10    51.046753    involved    51.859764   2.9911  -4.4963
707   Topic10    49.636219   equipment    50.448677   2.9906  -4.5243
827   Topic10    49.380817   receiving    50.193386   2.9906  -4.5295
1976  Topic10    47.091766        zone    47.904087   2.9898  -4.5769
712   Topic10    45

In [45]:
# Compute Perplexity """lower the better."""
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score  """ Higher is better """
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v', topn=30)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("Topics_Coherence, Standard_Deviation, Support")# support is the topn # of words in the topic
coherence_model_lda.get_coherence_per_topic(with_std=True, with_support=True)


Perplexity:  -8.647513628972279

Coherence Score:  0.2918708931920381
Topics_Coherence, Standard_Deviation, Support


[(0.24883883047397, 0.15070181177838352, 30),
 (0.3640212143096242, 0.1728460433295051, 30),
 (0.3043681213543857, 0.15733841675466081, 30),
 (0.20857023332639105, 0.13139820221784593, 30),
 (0.19762185365986615, 0.20209777243161234, 30),
 (0.27660346677784414, 0.3308920144832012, 30),
 (0.3579512635031015, 0.15742793158924182, 30),
 (0.28334842349039907, 0.17175016659713332, 30),
 (0.33473228126365034, 0.12873070535662512, 30),
 (0.342653243761149, 0.3221572141887927, 30)]

In [48]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8,
                                           random_state=100,
                                           chunksize=2,
                                           passes=10)#,
#                                            update_every=100,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.348555 -0.007401       1        1  30.729549
2     -0.040281 -0.325836       2        1  18.879519
1     -0.119413  0.209675       3        1  12.733978
6      0.030517  0.061197       4        1  11.644873
5      0.111238  0.037224       5        1   7.405361
4      0.118619 -0.033102       6        1   6.985628
7      0.133328  0.030871       7        1   6.164115
3      0.114548  0.027373       8        1   5.456971, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
181   Default  1507.000000      associate  1507.000000  30.0000  30.0000
230   Default   622.000000         stated   622.000000  29.0000  29.0000
72    Default   668.000000         pallet   668.000000  28.0000  28.0000
71    Default  1375.000000           pain  1375.000000  27.0000  27.0000
8     Default  1137.000000         amcare  1137.000000  26.0000  26.0000
267   Default  1488.000000       incident  1488.000000  25.0000  25.0000
1226  Default   417.000000        trailer   417.000000  24.0000  24.0000
294   Default   937.000000           time   937.000000  23.0000  23.0000
99    Default   924.000000          right   924.000000  22.0000  22.0000
162   Default   617.000000          shift   617.000000  21.0000  21.0000
118   Default   909.000000           work   909.000000  20.0000  20.0000
91    Default   796.000000       reported   796.000000  19.0000  19.0000
119   Default   726.000000        working   726.000000  18.0000  18.0000
144   Default   741.000000           left   741.000000  17.0000  17.0000
81    Default   398.000000        process   398.000000  16.0000  16.0000
145   Default   729.000000          level   729.000000  15.0000  15.0000
593   Default   307.000000           task   307.000000  14.0000  14.0000
424   Default   292.000000        trained   292.000000  13.0000  13.0000
385   Default   466.000000           knee   466.000000  12.0000  12.0000
909   Default   342.000000            aas   342.000000  11.0000  11.0000
138   Default   601.000000          floor   601.000000  10.0000  10.0000
291   Default   323.000000        station   323.000000   9.0000   9.0000
135   Default   739.000000           fell   739.000000   8.0000   8.0000
166   Default   544.000000      treatment   544.000000   7.0000   7.0000
2057  Default   221.000000            pmv   221.000000   6.0000   6.0000
198   Default   280.000000          first   280.000000   5.0000   5.0000
25    Default   402.000000            day   402.000000   4.0000   4.0000
612   Default   195.000000        october   195.000000   3.0000   3.0000
59    Default   323.000000        manager   323.000000   2.0000   2.0000
160   Default   473.000000         safety   473.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
318    Topic8   114.517227          found   115.298714   2.9015  -3.7869
673    Topic8    93.250206          cause    94.037933   2.8999  -3.9923
440    Topic8    72.302620           desk    73.085632   2.8975  -4.2468
681    Topic8    69.961006        factors    70.742805   2.8972  -4.2797
734    Topic8    66.330811        hazards    67.115639   2.8965  -4.3330
678    Topic8    53.796490  environmental    54.578449   2.8938  -4.5424
1551   Topic8    48.813732         loaded    49.601265   2.8923  -4.6396
687    Topic8    46.104542         normal    46.885887   2.8915  -4.6967
691    Topic8    41.960415           role    42.741585   2.8898  -4.7909
736    Topic8    40.404461            ice    41.185368   2.8891  -4.8287
1812   Topic8    37.493706       expected    38.316978   2.8866  -4.9035
384    Topic8    35.441433          issue    36.228245   2.8863  -4.9597
231    Topic8    33.232468     wheelchair    34.015400   2.8850  -5.0241
255    Topic8    34.159401       checking    34.972450   2.

By comparing the coherence perplexity scores above, we see that the model with 9 topics would be most appropriate for Falls at the same level. 

In [62]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9,
                                           random_state=100,
                                           chunksize=2,
                                           passes=10)#,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.414167 -0.016417       1        1  35.760040
3     -0.080442 -0.243344       2        1  11.426280
2     -0.025760 -0.000726       3        1   9.646547
4      0.014885 -0.039549       4        1   8.019679
1     -0.089258 -0.126762       5        1   7.971203
6     -0.072009 -0.006012       6        1   7.376476
8     -0.029989  0.156404       7        1   7.075877
7     -0.073734  0.205961       8        1   6.530879
5     -0.057861  0.070446       9        1   6.193013, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
181   Default  1641.000000   associate  1641.000000  30.0000  30.0000
72    Default   761.000000      pallet   761.000000  29.0000  29.0000
230   Default   627.000000      stated   627.000000  28.0000  28.0000
267   Default  1528.000000    incident  1528.000000  27.0000  27.0000
71    Default  1392.000000        pain  1392.000000  26.0000  26.0000
8     Default  1151.000000      amcare  1151.000000  25.0000  25.0000
1226  Default   420.000000     trailer   420.000000  24.0000  24.0000
118   Default   973.000000        work   973.000000  23.0000  23.0000
294   Default   949.000000        time   949.000000  22.0000  22.0000
99    Default   905.000000       right   905.000000  21.0000  21.0000
385   Default   414.000000        knee   414.000000  20.0000  20.0000
119   Default   755.000000     working   755.000000  19.0000  19.0000
291   Default   329.000000     station   329.000000  18.0000  18.0000
593   Default   311.000000        task   311.000000  17.0000  17.0000
91    Default   806.000000    reported   806.000000  16.0000  16.0000
81    Default   379.000000     process   379.000000  15.0000  15.0000
314   Default   291.000000        dock   291.000000  14.0000  14.0000
135   Default   754.000000        fell   754.000000  13.0000  13.0000
144   Default   750.000000        left   750.000000  12.0000  12.0000
145   Default   738.000000       level   738.000000  11.0000  11.0000
424   Default   343.000000     trained   343.000000  10.0000  10.0000
198   Default   281.000000       first   281.000000   9.0000   9.0000
162   Default   693.000000       shift   693.000000   8.0000   8.0000
501   Default   246.000000      proper   246.000000   7.0000   7.0000
404   Default   310.000000         ppe   310.000000   6.0000   6.0000
472   Default   316.000000        date   316.000000   5.0000   5.0000
138   Default   604.000000       floor   604.000000   4.0000   4.0000
160   Default   473.000000      safety   473.000000   3.0000   3.0000
115   Default   305.000000     tripped   305.000000   2.0000   2.0000
105   Default   293.000000        site   293.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
498    Topic9   105.306282         ops   106.103821   2.7742  -3.9973
1156   Topic9    93.204254   condition    94.002235   2.7732  -4.1194
2494   Topic9    86.975166    rotation    87.772453   2.7726  -4.1885
509    Topic9    77.622971     scanner    78.420738   2.7715  -4.3023
1651   Topic9    71.964600     regular    72.764824   2.7707  -4.3780
12     Topic9    71.295906   attention    72.093475   2.7706  -4.3873
517    Topic9    69.891304     treated    70.688591   2.7704  -4.4072
453    Topic9    67.858368       using    68.656433   2.7701  -4.4367
520    Topic9    60.744301      weight    61.541725   2.7687  -4.5475
2493   Topic9    60.324413     require    61.121849   2.7686  -4.5544
481    Topic9    54.684799    finished    55.482594   2.7673  -4.6526
478    Topic9    48.584705         ehs    49.381950   2.7655  -4.7708
494    Topic9    45.939495    involved    46.736633   2.7645  -4.8268
707    Topic9    44.676384   equipment    45.473984   2.7641  -4.8547
827    Topic9    44.470085   receiving    45.267529   2.7640  -4.8593
442

Get the the most probable topics for each document

In [63]:
topics = [lda_model.get_document_topics(doc_bow) for doc_bow in corpus]
topics

[[(0, 0.23153695),
  (1, 0.023153786),
  (2, 0.27133635),
  (3, 0.022114957),
  (4, 0.2313564),
  (5, 0.06715879),
  (6, 0.027198138),
  (7, 0.06684518),
  (8, 0.059299435)],
 [(0, 0.6938113),
  (2, 0.01773035),
  (3, 0.04598546),
  (4, 0.14200546),
  (5, 0.016680358),
  (6, 0.05868582),
  (7, 0.022357028)],
 [(0, 0.16250819), (1, 0.7213045), (2, 0.024902645), (3, 0.08298957)],
 [(0, 0.3378478),
  (1, 0.36166754),
  (6, 0.09607182),
  (7, 0.07336699),
  (8, 0.106337585)],
 [(0, 0.3413645), (1, 0.08820496), (2, 0.06538126), (6, 0.47979078)],
 [(0, 0.30264848),
  (1, 0.10472964),
  (2, 0.029442092),
  (4, 0.02858812),
  (5, 0.03129626),
  (6, 0.48440373),
  (8, 0.016421935)],
 [(0, 0.30456096),
  (1, 0.03734082),
  (2, 0.45307678),
  (3, 0.052343823),
  (5, 0.011415017),
  (6, 0.024726415),
  (8, 0.114439115)],
 [(0, 0.27331847),
  (1, 0.016953524),
  (2, 0.0864825),
  (3, 0.50595176),
  (4, 0.0658734),
  (6, 0.035379294)],
 [(0, 0.35968494),
  (1, 0.31567124),
  (3, 0.032311935),
  (4, 

Sort the topics in descending order so that the topic with the highest probability for each document shows up first and then get the topic Numbers.

In [64]:
sorted_topics = [sorted(doc_topics, key=lambda x: x[1], reverse =  True) for doc_topics in topics]
topic_num = [x[0][0] + 1  for x in sorted_topics]
print([(topic, topic_num.count(topic)) for topic in set(topic_num)])

[(1, 816), (2, 52), (3, 38), (4, 33), (5, 16), (6, 17), (7, 17), (8, 16), (9, 23)]


In [75]:
data['Body Part'].value_counts()

Knee                            226
Back                            135
Ankle                           116
Wrist                            99
Shoulder                         65
Hand                             46
Arm                              45
Hips/pelvis                      44
Elbow                            40
Head - Facial Area               37
Head other than face             36
Leg                              34
Abdomen/lower trunk/buttocks     28
Foot                             23
Finger                           13
Chest                            11
Trunk; upper                      7
Neck                              7
Eye                               5
Groin                             4
Toe                               4
Internal Systems/Organs           3
Name: Body Part, dtype: int64

In [72]:
data['Topics Dist'] = sorted_topics

In [71]:
sorted_topics[0]

[(2, 0.27133635),
 (0, 0.23153695),
 (4, 0.2313564),
 (5, 0.06715879),
 (7, 0.06684518),
 (8, 0.059299435),
 (6, 0.027198138),
 (1, 0.023153786),
 (3, 0.022114957)]

Add a predicted Topic for each document

In [65]:
data['PredictedTopic'] =['Topic {}'.format(topic) for index, topic in enumerate(topic_num)]

In [73]:
data.head()

,Organization,Sub-Organization,Site,System Case ID,Case Date,Incident Type,Body Part,Cause Object,Injury Type,Description,Descriptive Root Cause,Corrective Actions,Case Level,RawPrep,PredictedTopic,Topics Dist
0,Amazon Transportation Services,Sort Center - North America,MSP5,245,2017-11-14,"Falls, same level",Leg,Equip: Pallet,Fracture (Non-Hairline),"MSP5 (Right ankle fracture, slip/trip/fall, Pa...",AA did not have their on path or task while wo...,AA should get coached on taking two packages a...,Level B,"[msp5, right, ankle, fracture, slip, trip, fal...",Topic 3,"[(2, 0.27133635), (0, 0.23153695), (4, 0.23135..."
1,North America Fulfillment Centers (FCs),Returns,CVG2,2662,2017-11-15,"Falls, same level",Elbow,Debris: Other,Fracture (Non-Hairline),On 11/15/17 at approx 11:00 AA was working in ...,It was determined the AA unintentionally tripp...,Perform FSI with focus on lose banding. ASC me...,Level B,"[11/15/17, approx, 11:00, working, carpets, ar...",Topic 1,"[(0, 0.6938113), (4, 0.14200546), (6, 0.058685..."
2,Europe Fulfillment Centers (FCs),AR Sortable,MAN1,409,2017-11-14,"Falls, same level",Knee,NaN,Loss of consciousness,Amazon Associate (AA) has been staffed on AFE ...,AA is not feeling well.,AM to ensure that they inform AA if they are n...,Level B,"[amazon, associate, staffed, afe, empty, tote,...",Topic 2,"[(1, 0.7213045), (0, 0.16250819), (3, 0.082989..."
3,Amazon Transportation Services,Sort Center - North America,BOS5,612,2017-11-17,"Falls, same level",Shoulder,NaN,Loss of consciousness,AA was scanning on line G @ I-Beam G-16. AA pa...,"AA had been feeling ill during the day, they w...",none needed non-occ injury,Level B,"[scanning, line, beam, g-16, passed, injured, ...",Topic 2,"[(1, 0.36166754), (0, 0.3378478), (8, 0.106337..."
4,Amazon Transportation Services,Sort Center - North America,LAS5,53,2017-11-15,"Falls, same level",Head - Facial Area,NaN,Loss of consciousness,"On Nov 15, 2017, at approximately 0826 hours, ...",AA fell on the floor and hit their head.,To remind AA to be mindful of their surroundings.,Level B,"[nov, 2017, 0826, hours, splitting, lane, low,...",Topic 7,"[(6, 0.47979078), (0, 0.3413645), (1, 0.088204..."


In [69]:
import pprint
for topic in lda_model.show_topics(num_words=30, num_topics=9, formatted = False):
    print('Topic {}:'.format(topic[0] +1), '\n', ' | '.join([wordpair[0] for wordpair in topic[1]]), '\n', '--'*78)
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))

Topic 1: 
 incident | pain | amcare | work | time | right | reported | fell | left | level | shift | working | treatment | area | injury | floor | walking | day | fall | aas | felt | safety | manager | side | foot | slipped | went | ground | 2018 | investigation 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 2: 
 first | break | escorted | started | tote | aid | next | september | way | hospital | decided | doctor | aider | amazon | informed | sos | cart | asked | steady | wanted | ready | scanning | twisted | office | responded | stairs | leave | applied | paramedic | come 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 3: 
 pallet | lower | recordable | making | outside | knees | osha | case | top | stand | see | restrictions | bin | balance | lane | me

In [70]:
l9 = []
for topic in lda_model.show_topics(num_words=20, num_topics=9, formatted = False):
    l9.append(('Topic {}:'.format(topic[0] +1), ' | '.join([wordpair[0] for wordpair in topic[1]])))
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))
pd.DataFrame(l9, columns= 'TopicNum KeyWords'.split()).to_excel('Topics9FallsSameLevel.xlsx', index = False)

Another way to visualize text is to use the displacy method of SpaCy. It will show all NAMED ENTITIES in the text.

In [216]:
from spacy import displacy
displacy.render(nlp("""The author of the age of reason is Thomas Paine. He was an aethiest. Because of this, 
                    people did not like him. They thought he was racist but maybe he was just ahead of his contemporaries."""), jupyter=True, style='ent')

In [220]:
displacy.render(nlp('''On 5/21/18 at 1730, AA was offstacking on slam line 4. The AA hung up a jam clearing device on the metal grate that covers the nip point between 2 conveyors. The location the AA hung the device on was not 5s. Right after the AA hung up the device, the device fell and struck them on the back of their right arm, right above the back of their elbow. The AA reported to AMCARE immediately after with a pain level of 5 at the time of the incident. The AA reported a pain level of 2 after treatment. The AA is currently up to date on their trainings, they were not being labor shared, they were not on overtime, they were wearing their PPE, there was no equipment involved, and the AA was able to return back to their home path without any modifications.'''), 
                jupyter=True, style = 'ent')